<a href="https://colab.research.google.com/github/takahashi0329/DeepLerarning2022Late/blob/main/%E5%8F%8E%E9%9B%86%E3%81%97%E3%81%9F%E7%94%BB%E5%83%8F%E3%81%A7%E5%AD%A6%E7%BF%92%E3%81%A8%E5%88%86%E9%A1%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 収集した画像をColabにUPして学習させ、それをもとに分類させる

##ファイルのフォルダを準備する

In [1]:
import os
os.makedirs('sample', exist_ok=True) #分類対象の画像のフォルダを作成
os.makedirs('output/dog/sample', exist_ok=True) #犬と判定された時の置き場所
os.makedirs('output/cat/sample', exist_ok=True) #猫と判定された時の置き場所
os.makedirs('img/deep_learning/dog', exist_ok=True) #犬の学習用の収集画像
os.makedirs('img/deep_learning/cat', exist_ok=True) #猫の学習用の収集画像

## 必要なライブラリをimport

In [2]:
import cv2
import numpy as np
import glob as glob
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

##分類（クラス）の名前をフォルダ名（dog,cat）から引用する

In [3]:
path = "img/deep_learning" # dogとcatフォルダがある場所
folders = os.listdir(path)
# 2つのパスからフォルダを抜き出す
classes = [f for f in folders if os.path.isdir(os.path.join(path, f))]
print(classes)
n_classes = len(classes)
print(n_classes)

['cat', 'dog']
2


## 画像読み込みリサイズ、整形する

In [4]:
# 画像とラベルのための配列
X = [] #画像（学習データ用の配列）
Y = [] #ラベル（正解の配列）

# 画像を読み込みつつリサイズ
for label, class_name in enumerate(classes):
  files = glob.glob(path + "/" + class_name + "/*jpg")
  for file in files:
    img = cv2.imread(file)
    img = cv2.resize(img, dsize=(224,224))
    X.append(img)
    Y.append(label)
    # 内部ループ終わり
# 外部ループ終わり

## 0～255のビットデータ値を、学習のために0～1に変換(正規化)学習精度を上げる

In [5]:
X = np.array(X) # 配列XをNumpy配列に変換
X = X.astype('float32') #小数型に変換
X /=255.0
# ラベルもnumpyの配列に変換
Y = np.array(Y)
Y = np_utils.to_categorical(Y, n_classes)

### 整形したデータを学習用とテスト検証用にふりわける

In [6]:
# 学習データ8割、テスト検証用が2割にわける
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
# 学習データ8割
print(X_train.shape)
# テストデータ2割
print(X_test.shape)
# 学習データ8割(正解ラベル)
print(Y_train.shape)
# テストデータ2割(正解ラベル)
print(Y_test.shape)

(120, 224, 224, 3)
(31, 224, 224, 3)
(120, 2)
(31, 2)


## 準備したデータを学習させていく

###学習に必要なライブラリをimport

In [7]:
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input, Activation, Dense, Flatten, Dropout
from keras.optimizers import Adam

##モデルのVGG16をアレンジしていく

In [8]:
# VGG16 入力層を用意する（定義する）
input_tensor = Input(shape=(224, 224, 3))
# VGG16のインスタンスを作る（最後のsofmaxの1000層を無効にして省く）
base_model = VGG16(weights='imagenet', input_tensor=input_tensor, include_top=False)

58889256/58889256 [==============================] - 0s 0us/step


## 1000分類のsoftmaxを外した代わりに、2つに分類するsoftmax層を追加する

In [9]:
# 追加する層の作成
top_model = Sequential() #ラッピングする層
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(n_classes, activation='softmax'))

###最後のSOFTMAXを外したモデル(basemodel)と追加用の2つに分類するSOFTMAX層のモデル(top_model)を繋げる

In [10]:
model = Model(inputs=base_model.input, outputs=top_model(base_model.output))

###時間短縮のために、15層まで外す

In [11]:
#ループしながら15層までを無効にする
for layer in model.layers[:15]:
  layer.trainable = False #15層までは学習しない
#ループ終わり
print('# laers=', len(model.layers)) #モデルの層を表示

# laers= 20


##学習モデルをコンパイルする

In [12]:
# 損失はクロスエントロピー法で算出、最適化はADAM、指標は精度でみる
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

In [13]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

## コンパイルしたモデルで収集した画像データを学習させる

In [14]:
model.fit(X_train,Y_train,epochs=20,batch_size=16)

Epoch 1/20
8/8 [==============================] - 68s 8s/step - loss: 2.7756 - accuracy: 0.5583
Epoch 2/20
8/8 [==============================] - 78s 10s/step - loss: 1.0219 - accuracy: 0.5333
Epoch 3/20
8/8 [==============================] - 83s 10s/step - loss: 0.6971 - accuracy: 0.5250
Epoch 4/20
8/8 [==============================] - 69s 8s/step - loss: 0.7262 - accuracy: 0.5000
Epoch 5/20
8/8 [==============================] - 68s 8s/step - loss: 0.7057 - accuracy: 0.5500
Epoch 6/20
8/8 [==============================] - 70s 9s/step - loss: 0.6933 - accuracy: 0.4667
Epoch 7/20
8/8 [==============================] - 68s 8s/step - loss: 0.6932 - accuracy: 0.5000
Epoch 8/20
8/8 [==============================] - 69s 9s/step - loss: 0.6932 - accuracy: 0.5000
Epoch 9/20
8/8 [==============================] - 68s 8s/step - loss: 0.6932 - accuracy: 0.5000
Epoch 10/20
8/8 [==============================] - 68s 8s/step - loss: 0.6932 - accuracy: 0.5000
Epoch 11/20
8/8 [====================

##学習の成果を指標(accuracy)で確認

In [15]:
score = model.evaluate(X_test, Y_test, batch_size=16)

2/2 [==============================] - 14s 7s/step - loss: 0.6932 - accuracy: 0.4839


### 作成した学習モデルとクラス分類名を保存

In [16]:
import pickle
# vクラス分類の保存
pickle.dump(classes, open('classes.sav', 'wb')) # オブジェクトをバイナリファイルで保存
# 学習モデル保存
model.save('cnn.h5')

##保存したクラス分類と学習モデルを読み込んで使う

In [17]:
# ライブラリのimport
from keras.models import load_model
import pickle
import cv2
import glob

##ファイルを読み込んでオブジェクトとしてよみがえらせる

In [18]:
model = load_model('cnn.h5')
classes = pickle.load(open('classes.sav', 'rb'))

### Sample画像を読み込む

In [21]:
files = glob.glob('sample/*')
print(files)

['sample/dog_0.jpg', 'sample/cat_4.jpg', 'sample/cat_13.jpg']


## ファイルパスから画像を読み込み、学習モデルに判定させ、該当するoutputフォルダにコピーを書き込む

In [23]:
for file in files:#ファイルの数だけループ
  img = cv2.imread(file) # 画像データを読み込み
  # 判定のためにimgを加工したimg2を作る
  img2 = cv2.resize(img, dsize=(224, 224)) # (224, 224)にリサイズ
  img2 = img2.astype('float32')
  img2 = img2/255.0 #0～255.0 だったデータを0.0～1.0に変換
  img2 = img2[None, ...] #1次元配列に変換
  # 正規化したデータを判定
  result = model.predict(img2)
  print(result) #画像に生で出力
  # 確率が高いクラスを取得
  pred = result.argmax()
  print(pred, str(classes[pred]))
  # 判定したクラス分類のフォルダのほうを書き込む
  cv2.imwrite('output/' + str(classes[pred]) + '/' + file, img)
# ループ終わり

1/1 [==============================] - 0s 481ms/step
[[0.5005735  0.49942648]]
0 cat
1/1 [==============================] - 0s 474ms/step
[[0.5005735  0.49942648]]
0 cat
1/1 [==============================] - 0s 497ms/step
[[0.5005735  0.49942648]]
0 cat
